In [8]:
import pandas as pd
import numpy as np
import json
import networkx as nx
from tqdm import tqdm

In [9]:
graph = "memoryalpha"
embeddings = "dogtag_bgelarge"

In [10]:
embeddings_file = "./_input/node_embeddings/" + embeddings + "/" + graph + ".json"
graph_file = "_input/triples/" + graph + ".triples"
mappings_file = "./_input/mappings/" + graph + ".json"
neighborhood_embeddings_file = "./_input/neighborhood_embeddings/" + embeddings + "/" + graph + ".json"

In [11]:
with open(mappings_file) as file:
    mappings = json.load(file)
    mappings_reversed = {str(v): k for k, v in mappings.items()}

In [12]:
df = pd.read_json(embeddings_file)
df_T = df.T

In [13]:
graph_df = pd.read_csv(graph_file, sep="###", engine="python", header=None)
G_nx = nx.Graph()
for row in graph_df.itertuples():
    G_nx.add_edge(int(row[1]), int(row[2]))
print(G_nx)

Graph with 48842 nodes and 1263597 edges


In [14]:
embeddings_dict = dict()

for n in tqdm(G_nx.nodes):
    
    neighbors = nx.neighbors(G_nx, int(n))
    embeddings = []

    selected_embedding = df_T.loc[mappings_reversed[str(n)]]
    embeddings.append(selected_embedding)

    for node in neighbors:
        selected_embedding = df_T.loc[mappings_reversed[str(node)]]
        embeddings.append(selected_embedding)

    neighborhood_embedding = np.average(embeddings, axis=0)
    embeddings_dict.update({n: list(neighborhood_embedding)})

with open(neighborhood_embeddings_file, "w") as json_file:
    json.dump(embeddings_dict, json_file, indent=4)

100%|██████████| 48842/48842 [00:52<00:00, 935.22it/s] 
